In [1]:
import os
import shutil
from tqdm import tqdm

def organize_files(src_root_dir, dst_root_dir):
    cases = ['case1', 'case2', 'case3', 'case4']

    # Create directories for each case
    for case in cases:
        case_dir = os.path.join(dst_root_dir, case)
        if not os.path.exists(case_dir):
            os.makedirs(case_dir)

    # Traverse the source directory
    for main_dir in tqdm(os.listdir(src_root_dir), desc="Processing directories"):
        main_dir_path = os.path.join(src_root_dir, main_dir)
        if os.path.isdir(main_dir_path):
            for sub_dir in os.listdir(main_dir_path):
                sub_dir_path = os.path.join(main_dir_path, sub_dir)
                if os.path.isdir(sub_dir_path):
                    case_name = sub_dir.split('_')[0]
                    case_dir = os.path.join(dst_root_dir, case_name)

                    # 首先复制原始的case1_1.wav文件
                    if case_name == 'case1':
                        original_wav = os.path.join(sub_dir_path, f"{main_dir}_{sub_dir}.wav")
                        if os.path.exists(original_wav):
                            dst_original_wav = os.path.join(case_dir, f"{main_dir}_{sub_dir}.wav")
                            shutil.copy(original_wav, dst_original_wav)

                    # 然后复制分割后的音频和对应的npy文件
                    for filename in os.listdir(sub_dir_path):
                        if filename.startswith('sample_') and (filename.endswith('.wav') or filename.endswith('.npy')):
                            src_path = os.path.join(sub_dir_path, filename)
                            dst_path = os.path.join(case_dir, f"{main_dir}_{sub_dir}_{filename}")
                            shutil.copy(src_path, dst_path)
src_directory = 'dataset_2k3k_withbandpass_extrafeatures_v3'
dst_directory = 'data_2k3k_nobandpass_organized_dataset_extrafeatures'
    
print(f"源目录: {src_directory}")
print(f"目标目录: {dst_directory}")

if not os.path.exists(src_directory):
    print(f"源目录不存在: {src_directory}")
    
organize_files(src_directory, dst_directory)
print("文件整理完成。")

源目录: dataset_2k3k_withbandpass_extrafeatures_v3
目标目录: data_2k3k_nobandpass_organized_dataset_extrafeatures


Processing directories: 100%|██████████| 19/19 [00:16<00:00,  1.17it/s]

文件整理完成。


In [2]:
import os
import shutil

# 定义路径
root_path = 'data_2k3k_nobandpass_organized_dataset_extrafeatures'
new_structure_path = 'data_2k3k_nobandpass_organized_withoutA3A7_dataset'

# 创建新的目录结构
if not os.path.exists(new_structure_path):
    os.makedirs(new_structure_path)

# 遍历case1到case4
for case in ['case1', 'case2', 'case3', 'case4']:
    case_path = os.path.join(root_path, case)
    for root, dirs, files in os.walk(case_path):
        for file in files:
            if file.endswith('.wav'):
                # 解析文件名获取前缀、case_id和sample_set
                parts = file.split('_')
                prefix = parts[0]  # 获取前缀部分，如A1

                # 跳过prefix为"A3"或"A7"的文件
                if prefix in ['A3', 'A7']:
                    continue

                case_id = f"{parts[1]}_{parts[2]}"
                sample_set = parts[3]
                sample_id = parts[4].split('.')[0]  # 获取sample集编号

                # 创建新的路径
                new_case_path = os.path.join(new_structure_path, case)
                if not os.path.exists(new_case_path):
                    os.makedirs(new_case_path)

                new_prefix_path = os.path.join(new_case_path, prefix)
                if not os.path.exists(new_prefix_path):
                    os.makedirs(new_prefix_path)

                new_case_id_path = os.path.join(new_prefix_path, case_id)
                if not os.path.exists(new_case_id_path):
                    os.makedirs(new_case_id_path)

                new_sample_set_path = os.path.join(new_case_id_path, f'sample_{sample_set}')
                if not os.path.exists(new_sample_set_path):
                    os.makedirs(new_sample_set_path)

                # 移动文件到新的路径
                old_file_path = os.path.join(root, file)
                new_file_path = os.path.join(new_sample_set_path, file)
                shutil.move(old_file_path, new_file_path)

                npy_file = file.replace('.wav', '.npy')
                old_npy_path = os.path.join(root, npy_file)
                if os.path.exists(old_npy_path):  # 检查npy文件是否存在
                    new_npy_path = os.path.join(new_sample_set_path, npy_file)
                    shutil.move(old_npy_path, new_npy_path)

print("文件重新组织完成。")


文件重新组织完成。


In [3]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split
import os
import sys

In [4]:
import os
from pathlib import Path
import torchaudio
from tqdm import tqdm

data = []

for case in ['case1', 'case2', 'case3', 'case4']:
    case_path = Path(f'data_2k3k_nobandpass_organized_withoutA3A7_dataset/{case}')
    for path in tqdm(case_path.glob("**/*.wav")):
        name = path.stem
        # 获取前缀、case_id和sample_set
        parts = path.parts[-4:]  # 获取最后4部分: case文件夹, 前缀文件夹, case_id文件夹, 和文件名
        prefix = parts[1]  # 前缀文件夹
        case_id = parts[2]  # case_id文件夹
        sample_set = parts[3].split('_')[1]  # 从文件名中提取sample_set

        try:
            # 加载文件
            s = torchaudio.load(path)
             # 加载对应的 .npy 文件
            npy_path = path.with_suffix('.npy')
            if npy_path.exists():
                energy_features = np.load(npy_path)
            else:
                energy_features = None
                print(f"Warning: No .npy file found for {path}")
            data.append({
                "name": name,
                "path": str(path),
                "case": case,
                "prefix": prefix,
                "case_id": case_id,
                "sample_set": sample_set,
                "energy_features": energy_features
            })
        except Exception as e:
            # 跳过损坏的文件
            pass

# 显示收集到的数据条目数
print(f"Collected {len(data)} items.")


456it [00:04, 92.16it/s] 
456it [00:04, 96.18it/s] 
912it [00:11, 81.47it/s]
912it [00:08, 101.99it/s]

Collected 2736 items.


In [5]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,name,path,case,prefix,case_id,sample_set,energy_features
0,A1_case1_1_sample_10_2,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case1,case1_1,sample_sample,case1,0.03039726577755708
1,A1_case1_1_sample_11_2,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case1,case1_1,sample_sample,case1,0.03030826823980881
2,A1_case1_1_sample_12_2,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case1,case1_1,sample_sample,case1,0.026308367936348276
3,A1_case1_1_sample_1_1,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case1,case1_1,sample_sample,case1,0.018682080938106083
4,A1_case1_1_sample_2_1,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case1,case1_1,sample_sample,case1,0.021825769392396666


In [6]:
import os
import pandas as pd
import torch
import torchaudio
from scipy.io import wavfile
from scipy.signal import resample
from torchaudio.utils import download_asset
from IPython.display import Audio

# Define function to resample audio
def resample_audio(data, orig_sr, target_sr=16000):
    number_of_samples = round(len(data) * float(target_sr) / orig_sr)
    resampled_data = resample(data, number_of_samples)
    return resampled_data

# Load and process your dataframe `df`
file_path_column = "path"
df["status"] = df[file_path_column].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["status"])
df = df.drop("status", axis=1)

# Shuffle and reset index of dataframe
df = df.sample(frac=1).reset_index(drop=True)
print(df.head())  # Check the first few rows after preprocessing

# Process each audio file in the dataframe
for index, row in df.iterrows():
    file_path = row[file_path_column]
    orig_sample_rate, data = wavfile.read(file_path)

    # Resample audio to target sample rate (16000 Hz)
    resampled_data = resample_audio(data, orig_sample_rate, target_sr=16000)

    # Save the resampled data back to a file
    resampled_path = file_path.replace(".wav", "_resampled.wav")
    torchaudio.save(resampled_path, torch.from_numpy(resampled_data).unsqueeze(0), sample_rate=16000)

    # Update the path in the dataframe
    df.at[index, file_path_column] = resampled_path

    # Print progress or any other processing steps
    print(f"Processed {file_path} and saved resampled audio to {resampled_path}")

# Optionally, save the updated dataframe with processed data paths
df.to_csv("processed_audio_data.csv", index=False)


                     name                                               path  \
0   E1_case1_2_sample_1_1  data_2k3k_nobandpass_organized_withoutA3A7_dat...   
1   E6_case3_1_sample_2_1  data_2k3k_nobandpass_organized_withoutA3A7_dat...   
2  A9_case2_2_sample_12_1  data_2k3k_nobandpass_organized_withoutA3A7_dat...   
3  E1_case3_5_sample_29_2  data_2k3k_nobandpass_organized_withoutA3A7_dat...   
4  A1_case4_5_sample_30_2  data_2k3k_nobandpass_organized_withoutA3A7_dat...   

    case   prefix        case_id sample_set       energy_features  
0  case1  case1_2  sample_sample      case1  0.004840230967197101  
1  case3  case3_1  sample_sample      case3   0.29115209598610736  
2  case2  case2_2  sample_sample      case2   0.11847946405159782  
3  case3  case3_5  sample_sample      case3   0.07505743331469183  
4  case4  case4_5  sample_sample      case4    0.5185114064133831  
Processed data_2k3k_nobandpass_organized_withoutA3A7_dataset\case1\E1\case1_2\sample_sample\E1_case1_2_sample_1

Let's explore how many labels (emotions) are in the dataset with what distribution.

In [7]:
# Filter broken and non-existent paths

print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["status"])
df = df.drop("status", axis=1)
print(f"Step 1: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)

# Print unique emotions and count
print("Labels: ", df["case"].unique())
print()
print(df.groupby("case").count()[["path"]])


Step 0: 2736
Step 1: 2736
Labels:  ['case4' 'case3' 'case2' 'case1']

       path
case       
case1   456
case2   456
case3   912
case4   912


For training purposes, we need to split data into train test sets; in this specific example, we break with a `20%` rate for the test set.

In [8]:
import os
import pandas as pd
import torchaudio
import librosa
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from scipy.ndimage import maximum_filter1d, uniform_filter1d
from tqdm import tqdm
from pathlib import Path

save_path = "data_2k3k_nobandpass_organized_withoutA3A7_dataset"

# 添加一列来标识每个参与者
df['participant'] = df['name'].apply(lambda x: x.split('_')[0])

# 确保路径存在
df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["status"])
df = df.drop("status", axis=1)

# 获取所有参与者的唯一列表
participants = df['participant'].unique()

# 随机划分参与者
train_participants, eval_participants = train_test_split(participants, test_size=0.2, random_state=101)

# 根据参与者划分数据集
train_df = df[df['participant'].isin(train_participants)].reset_index(drop=True)
eval_df = df[df['participant'].isin(eval_participants)].reset_index(drop=True)

# 打印参与者信息以确认划分
print("Unique participants in training dataset:", train_df['participant'].unique())
print("Unique participants in evaluation dataset:", eval_df['participant'].unique())

# 保存为 CSV 文件
save_path = "data_2k3k_nobandpass_organized_withoutA3A7_dataset"
train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
eval_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)
# 打印数据集的形状
print(train_df.shape)
print(eval_df.shape)

# 加载数据集
from datasets import load_dataset

data_files = {
    "train": f"{save_path}/train.csv",
    "validation": f"{save_path}/test.csv",
}

dataset = load_dataset("csv", data_files={"train": f"{save_path}/train.csv", "validation": f"{save_path}/test.csv"}, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]


# 检查加载后的数据集
print("Loaded training dataset size:", len(train_dataset))
print("Loaded validation dataset size:", len(eval_dataset))

# 指定输入和输出列
input_column = "path"
output_column = "case"

# 加载能量特征
def load_energy_features(example):
    try:
        # 从 wav 文件路径获取对应的 npy 文件路径
        wav_path = example['path']
        npy_path = wav_path.replace('_resampled.wav', '.npy')
        
        # 加载 npy 文件
        features = np.load(npy_path)
        example['energy_features'] = features
        return example
    except Exception as e:
        print(f"Error loading energy features from {example}: {e}")
        example['energy_features'] = None
        return example

train_dataset = train_dataset.map(load_energy_features)
eval_dataset = eval_dataset.map(load_energy_features)

# 检查能量特征加载情况
print("Train dataset with energy features:")
print(train_dataset[:5])
print("Validation dataset with energy features:")
print(eval_dataset[:5])

# 打印数据集的前几行，检查数据完整性
print("Train dataset preview:")
print(train_dataset[:10])
print("Validation dataset preview:")
print(eval_dataset[:10])

# 打印每个 case 的样本数
print("Sample count per case in training dataset:")
print(train_dataset.to_pandas()[output_column].value_counts())
print("Sample count per case in validation dataset:")
print(eval_dataset.to_pandas()[output_column].value_counts())

# 识别和排序标签列表
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")


Unique participants in training dataset: ['E8' 'E4' 'A9' 'A4' 'E10' 'E6' 'A10' 'E5' 'A8' 'A1' 'E2' 'A5' 'E1' 'E9'
 'E3']
Unique participants in evaluation dataset: ['E11' 'A6' 'E7' 'A2']
(2160, 8)
(576, 8)


c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 2160 examples [00:00, 130209.21 examples/s]
Generating validation split: 576 examples [00:00, 143573.96 examples/s]


Loaded training dataset size: 2160
Loaded validation dataset size: 576


Map: 100%|██████████| 576/576 [00:00<00:00, 3601.45 examples/s]

Train dataset with energy features:
{'name': ['E8_case4_7_sample_39_2', 'E4_case4_2_sample_8_2', 'E8_case4_4_sample_24_1', 'A9_case3_11_sample_63_2', 'A4_case2_3_sample_16_2'], 'path': ['data_2k3k_nobandpass_organized_withoutA3A7_dataset\\case4\\E8\\case4_7\\sample_sample\\E8_case4_7_sample_39_2_resampled.wav', 'data_2k3k_nobandpass_organized_withoutA3A7_dataset\\case4\\E4\\case4_2\\sample_sample\\E4_case4_2_sample_8_2_resampled.wav', 'data_2k3k_nobandpass_organized_withoutA3A7_dataset\\case4\\E8\\case4_4\\sample_sample\\E8_case4_4_sample_24_1_resampled.wav', 'data_2k3k_nobandpass_organized_withoutA3A7_dataset\\case3\\A9\\case3_11\\sample_sample\\A9_case3_11_sample_63_2_resampled.wav', 'data_2k3k_nobandpass_organized_withoutA3A7_dataset\\case2\\A4\\case2_3\\sample_sample\\A4_case2_3_sample_16_2_resampled.wav'], 'case': ['case4', 'case4', 'case4', 'case3', 'case2'], 'prefix': ['case4_7', 'case4_2', 'case4_4', 'case3_11', 'case2_3'], 'case_id': ['sample_sample', 'sample_sample', 'sample_

In [9]:
# 统计包含能量特征的样本数
train_with_features = sum(1 for item in train_dataset if item['energy_features'] is not None)
eval_with_features = sum(1 for item in eval_dataset if item['energy_features'] is not None)
print(f"Train samples with energy features: {train_with_features} out of {len(train_dataset)}")
print(f"Validation samples with energy features: {eval_with_features} out of {len(eval_dataset)}")

Train samples with energy features: 2160 out of 2160
Validation samples with energy features: 576 out of 576


## Prepare Data for Training

In [10]:
# Loading the created dataset using datasets
from datasets import load_dataset

# 定义数据文件路径
data_files = {
    "train": "data_2k3k_nobandpass_organized_withoutA3A7_dataset/train.csv",
    "validation": "data_2k3k_nobandpass_organized_withoutA3A7_dataset/test.csv",
}

# 加载数据集
dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

# 打印数据集信息
print(train_dataset)
print(eval_dataset)


Dataset({
    features: ['name', 'path', 'case', 'prefix', 'case_id', 'sample_set', 'energy_features', 'participant'],
    num_rows: 2160
})
Dataset({
    features: ['name', 'path', 'case', 'prefix', 'case_id', 'sample_set', 'energy_features', 'participant'],
    num_rows: 576
})


In [11]:
input_column = "path"
output_column = "case"

In [12]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 4 classes: ['case1', 'case2', 'case3', 'case4']


In order to preprocess the audio into our classification model, we need to set up the relevant Wav2Vec2 assets regarding our language in this case `lighteternal/wav2vec2-large-xlsr-53-greek` fine-tuned by [Dimitris Papadopoulos](https://huggingface.co/lighteternal/wav2vec2-large-xlsr-53-greek). To handle the context representations in any audio length we use a merge strategy plan (pooling mode) to concatenate that 3D representations into 2D representations.

There are three merge strategies `mean`, `sum`, and `max`. In this example, we achieved better results on the mean approach. In the following, we need to initiate the config and the feature extractor from the Dimitris model.

In [13]:
from transformers import AutoConfig, Wav2Vec2Processor

In [14]:
model_name_or_path = "c3f9d884181a224a6ac87bf8885c84d1cff3384f"
pooling_mode = "mean"

In [15]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

setattr(config, 'use_energy_features', True)
setattr(config, 'energy_feature_dim', 1)

In [16]:
# processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
from transformers import Wav2Vec2FeatureExtractor
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
# target_sampling_rate = processor.feature_extractor.sampling_rate
target_sampling_rate = 16000
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


# Preprocess Data

So far, we downloaded, loaded, and split the SER dataset into train and test sets. The instantiated our strategy configuration for using context representations in our classification problem SER. Now, we need to extract features from the audio path in context representation tensors and feed them into our classification model to determine the emotion in the speech.

Since the audio file is saved in the `.wav` format, it is easy to use **[Librosa](https://librosa.org/doc/latest/index.html)** or others, but we suppose that the format may be in the `.mp3` format in case of generality. We found that the **[Torchaudio](https://pytorch.org/audio/stable/index.html)** library works best for reading in `.mp3` data.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a **map(...)** function accordingly. Also, we need to handle the string labels into integers for our specific classification task in this case, the **single-label classification** you may want to use for your **regression** or even **multi-label classification**.

In [17]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

# def preprocess_function(examples):
#     speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
#     target_list = [label_to_id(label, label_list) for label in examples[output_column]]

#     result = processor(speech_list, sampling_rate=target_sampling_rate)
#     result["labels"] = list(target_list)

#     return result

In [18]:
df.head(2)

,name,path,case,prefix,case_id,sample_set,energy_features,participant
0,E8_case4_7_sample_39_2,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case4,case4_7,sample_sample,case4,0.4646505166984447,E8
1,E11_case3_11_sample_65_2,data_2k3k_nobandpass_organized_withoutA3A7_dat...,case3,case3_11,sample_sample,case3,0.09038965620119965,E11


In [19]:
from transformers import Wav2Vec2FeatureExtractor

processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
# 定义输入和输出列
input_column = "path"
output_column = "case"

# 定义预处理函数
def preprocess_function(examples):
    # 处理音频数据
    audio = [torchaudio.load(path)[0].numpy().squeeze() for path in examples["path"]]
    result = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    result["labels"] = [label_list.index(label) for label in examples["emotion"]]
    
    # 加载能量特征
    energy_features = []
    for wav_path in examples["path"]:
        try:
            # 从wav文件路径构造npy文件路径
            npy_path = wav_path.replace('_resampled.wav', '.npy')
            if os.path.exists(npy_path):
                features = np.load(npy_path)
                energy_features.append(features)
            else:
                print(f"Warning: No .npy file found at {npy_path}")
                energy_features.append(None)
        except Exception as e:
            print(f"Error loading energy features from {wav_path}: {e}")
            energy_features.append(None)
            
    result["energy_features"] = energy_features
    
    return result

# # 加载数据集
# data_files = {
#     "train": "data_1k2k3k_nobandpass_organized_withoutA3A7_dataset/train.csv",
#     "validation": "data_1k2k3k_nobandpass_organized_withoutA3A7_dataset/test.csv",
# }
# dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
# train_dataset = dataset["train"]
# eval_dataset = dataset["validation"]

# 重命名 case 字段为 emotion
train_dataset = train_dataset.rename_column("case", "emotion")
eval_dataset = eval_dataset.rename_column("case", "emotion")

# 获取独特的标签
label_list = train_dataset.unique("emotion")
label_list.sort()  # 排序以确保确定性
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

# 预处理和映射数据集
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=1,
    remove_columns=["case_id", "sample_set", "prefix"]
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=1,
    remove_columns=["case_id", "sample_set", "prefix"]
)

print("Train dataset:")
print(train_dataset)
print("\nEval dataset:")
print(eval_dataset)

A classification problem with 4 classes: ['case1', 'case2', 'case3', 'case4']


Map: 100%|██████████| 576/576 [00:05<00:00, 96.17 examples/s]

Train dataset:
Dataset({
    features: ['name', 'path', 'emotion', 'energy_features', 'participant', 'input_values', 'attention_mask', 'labels'],
    num_rows: 2160
})

Eval dataset:
Dataset({
    features: ['name', 'path', 'emotion', 'energy_features', 'participant', 'input_values', 'attention_mask', 'labels'],
    num_rows: 576
})


In [20]:
train_participants = train_dataset.unique("participant")
print("Unique participants in training dataset:", train_participants)

eval_participants = eval_dataset.unique("participant")
print("Unique participants in evaluation dataset:", eval_participants)

Unique participants in training dataset: ['E8', 'E4', 'A9', 'A4', 'E10', 'E6', 'A10', 'E5', 'A8', 'A1', 'E2', 'A5', 'E1', 'E9', 'E3']
Unique participants in evaluation dataset: ['E11', 'A6', 'E7', 'A2']


In [21]:
idx = 0
print(f"Training input_values: {train_dataset[idx]['input_values']}")
print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['emotion']}")

Training input_values: [0.0403844453394413, -0.029195081442594528, 0.14053544402122498, -0.3657539486885071, -0.448139488697052, -0.03907610476016998, -0.01151761133223772, -0.4390745460987091, -0.2428799271583557, 0.09293434768915176, -0.19318923354148865, -0.2520824372768402, 0.19267404079437256, 0.29990556836128235, 0.012503670528531075, -0.1491413712501526, 0.048612117767333984, -0.03632141277194023, -0.5391668081283569, -0.4920955300331116, -0.022659670561552048, -0.1643109917640686, -0.5414825677871704, -0.25018107891082764, 0.570544421672821, 1.1173272132873535, 0.5286136269569397, -0.26706719398498535, -0.2181890457868576, 0.4691697657108307, 0.6100124716758728, 0.4932047128677368, 0.5642231702804565, 0.45773935317993164, 0.008795376867055893, 0.04247073829174042, 0.5225919485092163, 0.5888583660125732, 0.34911856055259705, 0.0189224686473608, 0.2271483987569809, 0.9311886429786682, 0.8069924116134644, -0.15434430539608002, -0.3001165986061096, 0.5433924198150635, 0.81943333148

Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

## Model

Before diving into the training part, we need to build our classification model based on the merge strategy.

In [22]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [23]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size + config.energy_feature_dim, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, energy_features=None, **kwargs):
        x = features
        if energy_features is not None:
            x = torch.cat([x, energy_features], dim=-1)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")
        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            energy_features=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        # print(f"Hidden states shape before: {hidden_states.shape}")
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        # print(f"Hidden states shape after: {hidden_states.shape}")

        logits = self.classifier(hidden_states, energy_features)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [24]:
# import torch
# import torch.nn as nn
# from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

# from transformers.models.wav2vec2.modeling_wav2vec2 import (
#     Wav2Vec2PreTrainedModel,
#     Wav2Vec2Model
# )

# class Wav2Vec2ClassificationHeadWithEnergy(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         # 修改这里的维度
#         self.energy_dense = nn.Linear(config.energy_feature_dim, 128)  # 改为固定输出128维
#         self.energy_activation = nn.ReLU()

#         # 修改输入维度：wav2vec2输出维度(1024) + energy特征维度(128)
#         self.dense = nn.Linear(config.hidden_size + 128, config.hidden_size)  # wav2vec2默认输出是768维

#         self.dropout = nn.Dropout(config.final_dropout)
#         self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

#     def forward(self, wav_features, energy_features=None):
#         if energy_features is not None:
#             e = self.energy_dense(energy_features)   # 映射到128维
#             e = self.energy_activation(e)
#             x = torch.cat([wav_features, e], dim=-1) 
#         else:
#             x = wav_features

#         x = self.dropout(x)
#         x = self.dense(x)
#         x = torch.tanh(x)
#         x = self.dropout(x)
#         x = self.out_proj(x)

#         return x

# class Wav2Vec2ForSpeechClassificationWithEnergy(Wav2Vec2PreTrainedModel):
#     def __init__(self, config):
#         super().__init__(config)
#         self.num_labels = config.num_labels
#         self.pooling_mode = config.pooling_mode
#         self.config = config

#         # 主体：Wav2Vec2 base
#         self.wav2vec2 = Wav2Vec2Model(config)
#         # 分头：使用带 energy MLP 的分类头
#         self.classifier = Wav2Vec2ClassificationHeadWithEnergy(config)

#         self.init_weights()

#     def freeze_feature_extractor(self):
#         self.wav2vec2.feature_extractor._freeze_parameters()

#     def merged_strategy(self, hidden_states, mode="mean"):
#         if mode == "mean":
#             outputs = torch.mean(hidden_states, dim=1)
#         elif mode == "sum":
#             outputs = torch.sum(hidden_states, dim=1)
#         elif mode == "max":
#             outputs = torch.max(hidden_states, dim=1)[0]
#         else:
#             raise ValueError("pooling_mode must be one of ['mean', 'sum', 'max']")
#         return outputs

#     def forward(
#         self,
#         input_values,
#         attention_mask=None,
#         energy_features=None,           # <--- 新增参数
#         output_attentions=None,
#         output_hidden_states=None,
#         return_dict=None,
#         labels=None,
#     ):
#         return_dict = return_dict if return_dict is not None else self.config.use_return_dict

#         # 1) 先跑 Wav2Vec2
#         outputs = self.wav2vec2(
#             input_values,
#             attention_mask=attention_mask,
#             output_attentions=output_attentions,
#             output_hidden_states=output_hidden_states,
#             return_dict=return_dict,
#         )

#         # 2) 取最后一层隐藏状态 (B, T, hidden_size)
#         hidden_states = outputs[0]

#         # 3) 在时间维度上做 pooling (mean/sum/max)
#         hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
#         # 此时 hidden_states: [batch_size, hidden_size]

#         # 4) 把 pooled hidden + energy_features (若有) 做融合并分类
#         logits = self.classifier(hidden_states, energy_features)

#         # 5) 算 loss (可选)
#         loss = None
#         if labels is not None:
#             if self.config.problem_type is None:
#                 if self.num_labels == 1:
#                     self.config.problem_type = "regression"
#                 elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
#                     self.config.problem_type = "single_label_classification"
#                 else:
#                     self.config.problem_type = "multi_label_classification"

#             if self.config.problem_type == "regression":
#                 loss_fct = MSELoss()
#                 loss = loss_fct(logits.view(-1, self.num_labels), labels)
#             elif self.config.problem_type == "single_label_classification":
#                 loss_fct = CrossEntropyLoss()
#                 loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             elif self.config.problem_type == "multi_label_classification":
#                 loss_fct = BCEWithLogitsLoss()
#                 loss = loss_fct(logits, labels)

#         if not return_dict:
#             output = (logits,) + outputs[2:]
#             return ((loss,) + output) if loss is not None else output

#         return SpeechClassifierOutput(
#             loss=loss,
#             logits=logits,
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#         )

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [25]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]
        energy_features = [feature.get("energy_features") for feature in features]
        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)
        batch["energy_features"] = torch.tensor(np.array(energy_features), dtype=torch.float)

        return batch

In [26]:
from transformers import Wav2Vec2FeatureExtractor

processor = Wav2Vec2FeatureExtractor.from_pretrained('c3f9d884181a224a6ac87bf8885c84d1cff3384f')
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
config.energy_feature_dim = 1

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

In [27]:
is_regression = False

In [28]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained XLSR-Wav2Vec2 checkpoint into our classification model with a pooling strategy.

In [29]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at c3f9d884181a224a6ac87bf8885c84d1cff3384f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [30]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [31]:
# from google.colab import drive

# drive.mount('/gdrive')

In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="train_result/123k_extrafeature",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    seed=2003,
)

c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
# !pip uninstall transformers[torch] -y
# !pip install transformers[torch]

In [34]:
# !pip uninstall transformers -y
# !pip install transformers


For future use we can create our training script, we do it in a simple way. You can add more on you own.

In [35]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_cuda_amq:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_cuda_amq:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


Now, all instances can be passed to Trainer and we are ready to start training!

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # tokenizer=processor.feature_extractor,
)


### Training

Training will take between 10 and 60 minutes depending on the GPU allocated to this notebook.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

\\```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [37]:
trainer.train()

  0%|          | 0/270 [00:00<?, ?it/s]c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:862: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECU

{'loss': 1.3659, 'grad_norm': 4.763462066650391, 'learning_rate': 9.62962962962963e-05, 'epoch': 0.04}


                                                
  4%|▎         | 10/270 [00:22<01:38,  2.63it/s] 

{'eval_loss': 1.3503739833831787, 'eval_accuracy': 0.3333333432674408, 'eval_runtime': 16.2931, 'eval_samples_per_second': 35.352, 'eval_steps_per_second': 8.838, 'epoch': 0.04}


  7%|▋         | 20/270 [00:38<02:57,  1.41it/s]

...

                                                 
100%|██████████| 270/270 [12:03<00:00,  1.91it/s]

{'eval_loss': 0.540112316608429, 'eval_accuracy': 0.6458333134651184, 'eval_runtime': 15.9371, 'eval_samples_per_second': 36.142, 'eval_steps_per_second': 9.036, 'epoch': 1.0}


100%|██████████| 270/270 [12:06<00:00,  2.69s/it]

{'train_runtime': 726.2727, 'train_samples_per_second': 2.974, 'train_steps_per_second': 0.372, 'train_loss': 0.8216899553934733, 'epoch': 1.0}


TrainOutput(global_step=270, training_loss=0.8216899553934733, metrics={'train_runtime': 726.2727, 'train_samples_per_second': 2.974, 'train_steps_per_second': 0.372, 'total_flos': 5.2502464438272e+16, 'train_loss': 0.8216899553934733, 'epoch': 1.0})

In [38]:
# trainer.evaluate()
trainer.evaluate(eval_dataset=eval_dataset)

100%|██████████| 144/144 [00:15<00:00,  9.20it/s]


{'eval_loss': 0.540112316608429,
 'eval_accuracy': 0.6458333134651184,
 'eval_runtime': 15.854,
 'eval_samples_per_second': 36.332,
 'eval_steps_per_second': 9.083,
 'epoch': 1.0}

In [39]:
trainer.evaluate(eval_dataset=train_dataset)

100%|██████████| 540/540 [01:00<00:00,  8.85it/s]


{'eval_loss': 0.5345504283905029,
 'eval_accuracy': 0.6504629850387573,
 'eval_runtime': 61.1595,
 'eval_samples_per_second': 35.317,
 'eval_steps_per_second': 8.829,
 'epoch': 1.0}

In [40]:
import torch
from collections import Counter
from transformers import Trainer, EvalPrediction
import numpy as np

class WeightedVoteTrainer(Trainer):
    def __init__(self, *args, num_votes=5, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_votes = num_votes

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        # 进行多次预测
        predictions = []
        confidences = []
        model.train()  # 启用 dropout
        for i in range(self.num_votes):
            torch.manual_seed(i)  # 为每次预测设置不同的随机种子
            inputs_copy = {k: v.clone() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
            
            # 添加小的随机噪声到输入
            if 'input_values' in inputs_copy:
                noise = torch.randn_like(inputs_copy['input_values']) * 0.01
                inputs_copy['input_values'] += noise
            
            _, logits, labels = super().prediction_step(model, inputs_copy, prediction_loss_only, ignore_keys)
            probs = torch.softmax(logits, dim=-1)
            confidence, pred = torch.max(probs, dim=-1)
            predictions.append(pred)
            confidences.append(confidence)
        
        model.eval()  # 恢复评估模式
        
        # 将预测结果和置信度堆叠起来
        stacked_preds = torch.stack(predictions, dim=0)
        stacked_confidences = torch.stack(confidences, dim=0)
        
        # 计算加权投票结果
        num_classes = logits.shape[-1]
        weighted_votes = torch.zeros((stacked_preds.shape[1], num_classes), device=stacked_preds.device)
        for i in range(self.num_votes):
            weighted_votes.scatter_add_(1, stacked_preds[i].unsqueeze(1), stacked_confidences[i].unsqueeze(1))
        
        weighted_vote_result = torch.argmax(weighted_votes, dim=1)

        return None, weighted_vote_result, labels

def compute_metrics_with_weighted_vote(eval_pred: EvalPrediction):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    
    # 确保预测和标签都是 NumPy 数组
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    # 计算准确率
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# 创建新的 Trainer 实例
weighted_vote_trainer = WeightedVoteTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics_with_weighted_vote,
    num_votes=6  # 设置投票次数
)

# 进行评估
eval_results = weighted_vote_trainer.evaluate(eval_dataset=eval_dataset)
print(eval_results)

100%|██████████| 144/144 [00:28<00:00,  5.13it/s]

{'eval_model_preparation_time': 0.0035, 'eval_accuracy': 0.6979166666666666, 'eval_runtime': 28.4164, 'eval_samples_per_second': 20.27, 'eval_steps_per_second': 5.067}
